# How to UsecKit

This notebook demonstrates the usage of the useckit library. Every output of this notebook will be placed in a folder named "_uscekit_out". To customize this behaviour change the output_dir attribute of the paradigm, evaluation method or prediction model instances.

## Create a dataset

The following snippet of code creates a dataset with samples of shape (100, 100, 4) and labels equal to "User_{x}". While the data samples need to be uniformly shaped numpy arrays of number types, the labels can be numpy arrays of any objects, for example strings.

In [ ]:
import sys, os
sys.path.append(os.path.dirname(os.getcwd()))  # fix jupyter path

from useckit import Dataset
from useckit.util.utils import make_some_intelligent_noise

x_train, y_train = make_some_intelligent_noise()
x_val, y_val = make_some_intelligent_noise()
x_test_enroll, y_test_enroll = make_some_intelligent_noise()
x_test_match, y_test_match = make_some_intelligent_noise()

dataset = Dataset(trainset_data=x_train,
                  trainset_labels=y_train,
                  validationset_data=x_val,
                  validationset_labels=y_val,
                  testset_enrollment_data=x_test_enroll,
                  testset_enrollment_labels=y_test_enroll,
                  testset_matching_data=x_test_match,
                  testset_matching_labels=y_test_match,
                  normalization_check=True)

## Create a Paradigm and evaluate it

In [ ]:
from useckit.paradigms.distance_learning.distance_paradigm import DistanceMetricParadigm

paradigm_1 = DistanceMetricParadigm()
paradigm_1.evaluate(dataset)

## Restore the trained model from the previous paradigm to evaluate it with different evaluation methods

In [ ]:
from useckit.paradigms.distance_learning.distance_paradigm import DistanceMetricParadigm
from useckit.paradigms.distance_learning.evaluation_methods.verification import Verification as DistanceVerification
from useckit.paradigms.distance_learning.evaluation_methods.identification_with_reject import IdentificationWithReject as DistanceIdentificationWithReject


saved_weights_path = os.path.join(paradigm_1.prediction_model.output_dir, 'best_model.hdf5')
paradigm_2 = DistanceMetricParadigm(evaluation_methods=[DistanceIdentificationWithReject(), DistanceVerification()])

We are not calling the evaluate method, hence we need to save the paradigm manually (if we so wish). This always needs to be done before fitting the paradigm's model or before restoring the paradigm's model's trainable values as we will be doing after.

In [ ]:
paradigm_2.persist()

Now we restore the paradigm's model's trainable values from the previous fitting during the evaluation of the first paradigm. Afterwards we run the evaluation methods of the paradigm with the already existing dataset. Note that we need to give the dataset as parameter for the restoration of the model as the paradigm uses it to infer among others the model's input layer size. Using a dataset with differently shaped samples will fail this process.

In [ ]:
paradigm_2.restore_prediction_model_trainables(saved_weights_path_like=saved_weights_path, dataset=dataset)
paradigm_2.run_evaluation_methods(dataset)

## Evaluate multiple paradigms with different prediction models to compare them amongst each other

Principally, we can combine any evaluation_method with any prediction_model from the respective sub-packages of the paradigms. Due to the single implementation of the different biometric tasks in the evaluation package, which is used by the evaluation_methods of all paradigms, the results are comparable not only within one, but across the paradigms as well. The below code snippet shows a small example of how to create three comparable experiments.

In [ ]:
from useckit.paradigms.time_series_classification.prediction_models.keras_model_descriptions import dl4tsc_mlp
from useckit.paradigms.time_series_classification.prediction_models.inception import dl4tsc_inception
from useckit.paradigms.time_series_classification.prediction_models.classification_keras_prediction_model import ClassificationKerasPredictionModel as TSCKerasModel
from useckit.paradigms.time_series_classification.evaluation_methods.identification import IdentificationOnly as TSCIdentification
from useckit.paradigms.time_series_classification.tsc_paradigm import TSCParadigm
from useckit.paradigms.anomaly_detection.prediction_models.auto_encoder_keras_prediction_model import AutoEncoderKerasPredictionModel as AnomalyKerasModel
from useckit.paradigms.anomaly_detection.evaluation_methods.identification import IdentificationOnly as AnomalyIdentification
from useckit.paradigms.anomaly_detection.anomaly_paradigm import AnomalyParadigm

x_train, y_train = make_some_intelligent_noise(shape=(40, 10, 10))
x_val, y_val = make_some_intelligent_noise(shape=(40, 10, 10))
x_test_enroll, y_test_enroll = make_some_intelligent_noise(shape=(40, 10, 10))
x_test_match, y_test_match = make_some_intelligent_noise(shape=(40, 10, 10))

dataset = Dataset(trainset_data=x_train,
                  trainset_labels=y_train,
                  validationset_data=x_val,
                  validationset_labels=y_val,
                  testset_enrollment_data=x_test_enroll,
                  testset_enrollment_labels=y_test_enroll,
                  testset_matching_data=x_test_match,
                  testset_matching_labels=y_test_match,
                  normalization_check=True)

paradigm_3 = TSCParadigm(evaluation_methods=[TSCIdentification()], prediction_model=dl4tsc_inception(nb_epochs=10))
paradigm_3.evaluate(dataset)

paradigm_4 = TSCParadigm(evaluation_methods=[TSCIdentification()],
                         prediction_model=TSCKerasModel(model_description=dl4tsc_mlp, nb_epochs=10))
paradigm_4.evaluate(dataset)

paradigm_5 = AnomalyParadigm(evaluation_methods=[AnomalyIdentification()], prediction_model=AnomalyKerasModel(nb_epochs=5))
paradigm_5.evaluate(dataset)

## Loading the exact configuration of a previously evaluated paradigm to recreate the experiment

In [ ]:
load_path = os.path.join(paradigm_4.output_dir, f"paradigm_{paradigm_4.name}.pickle")
paradigm_6 = TSCParadigm.restore(load_path, 'restored_paradigm', '_useckit_out_reruns')
paradigm_6.evaluate(dataset)

## UsecKit as part of a biometric system

The below example uses the TSCIdentificationModel from the time_series_classification.evaluation_methods package, but the principle is the same when working with the other Identification models or the IdentificationOrReject and Verification models implemented in the other paradigm's evaluation_methods packages.

In [ ]:
from useckit.paradigms.time_series_classification.evaluation_methods.identification import TSCIdentificationModel

identification_model = TSCIdentificationModel(dataset, paradigm_6.prediction_model)

Now we can identify additional samples using the identification model.

In [ ]:
import numpy as np

x_train, _ = make_some_intelligent_noise(labels=1, shape=(1, 10, 10))
sample = x_train[0]
print(identification_model.identify(np.array([sample])))

The below more complicated example shows how to do verification with the prediction model from paradigm 5. Note the extreme definition of the rejection thresholds, which will completely determine the result of the verification, no matter the actual predictions made.

In [ ]:
from useckit.paradigms.anomaly_detection.evaluation_methods.verification import AnomalyVerificationModel

verification_model = AnomalyVerificationModel(anomaly_prediction_model=paradigm_5.prediction_model,
                                              rejection_thresholds_per_trained_user=[10, 10, 10, 0])

all_identities = np.unique(dataset.gather_labels())
for i in all_identities:
    reverse_transformed_identity = dataset.reverse_label_transform(np.array([i]))
    verified = verification_model.verify(np.array([sample]), np.array([i]))
    print(f'The generated sample belongs to {reverse_transformed_identity}: {verified}')

In [ ]:
print("Notebook execution finished.")